<a href="https://colab.research.google.com/github/tiruvailshreya/Movie-Recommedations/blob/main/movie_regonition_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
!pip install scikit-learn
!pip install pandas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Load the MovieLens dataset
ratings_data = pd.read_csv('ratings.csv', names=['userId', 'movieId', 'rating', 'timestamp'])
movies_data = pd.read_csv('movies.csv', names=['movieId', 'title', 'genre'], header=0)
data = {
    'userId': [1, 2, 3, 4, 5],
    'movieId': [101, 102, 103, 104, 105],
    'rating': [4, 4.4, 3.8, 3, 5],
    'timestamp': ['9851648', '9658147', '694582', '9632584', '1254698']
}

ratings_data = pd.DataFrame(data)
# Create user-movie matrix
user_movie_matrix = ratings_data.pivot_table(index='userId', columns='movieId', values='rating', aggfunc='mean')

# Fill missing values with 0
user_movie_matrix = user_movie_matrix.fillna(0)
cosine_similarity_users = cosine_similarity(user_movie_matrix)
# Convert cosine similarity matrix into a DataFrame
cosine_similarity_users_df = pd.DataFrame(cosine_similarity_users, index=user_movie_matrix.index, columns=user_movie_matrix.index)

def get_movie_recommendations(user_id, top_n=5):
    # Find similar users
    similar_users = cosine_similarity_users_df.loc[user_id].sort_values(ascending=False)[1:]

    # Get movie ratings for similar users
    similar_users_ratings = user_movie_matrix.loc[similar_users.index]

    # Calculate weighted average ratings
    weighted_average_ratings = similar_users_ratings.apply(lambda x: x * similar_users, axis=0).sum()

    # Exclude already watched movies
    user_movies = user_movie_matrix.loc[user_id]
    recommendations = weighted_average_ratings.drop(user_movies[user_movies > 0].index)

    # Sort recommendations by rating and get top N
    top_recommendations = recommendations.sort_values(ascending=False).head(top_n)

    return top_recommendations
# Prompt the user to input a userID
user_id_input = input('Enter userID: ')

# Convert the user input to an integer if possible
try:
    user_id = int(user_id_input)
except ValueError:
    print('Invalid input! Please enter a valid userID.')
    exit()  # Exit the program if input is invalid

# Check if the user ID is less than 5
if user_id < 5:
    # Call get_movie_recommendations function
    top_recommendations = get_movie_recommendations(user_id, top_n=5)
    print(f"Top 5 movie recommendations for user {user_id}:")
    print(top_recommendations)
else:
    print('Invalid userID! Please enter a userID less than 5.')

# Call the function with user_id
get_movie_recommendations(user_id, top_n=5)



Enter userID: 2
Top 5 movie recommendations for user 2:
movieId
101    0.0
103    0.0
104    0.0
105    0.0
dtype: float64


movieId
101    0.0
103    0.0
104    0.0
105    0.0
dtype: float64